In [33]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.cluster import KMeans
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import enum

In [25]:
def tokenize(text):
    text = str(text)
    tokens = TweetTokenizer().tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha() and token not in set(stopwords.words("english"))]
    return tokens

In [26]:
df = pd.read_csv("all_posts.csv")
df["tokenized_selftext"] = df["selftext"].apply(tokenize)
df.head()

,id,subreddit_id,created_utc,title,selftext,ups,downs,tokenized_selftext
0,fzalyr,t5_2qwyh,1.586626e+09,growing up as a mentally ill person,like assume many others well issues mental ill...,94.0,0.0,"[like, assume, many, others, well, issues, men..."
1,nkmwur,t5_2qwyh,1.621940e+09,How to drag myself out of this darkness?,trying things productive take care soo long ne...,91.0,0.0,"[trying, things, productive, take, care, soo, ..."
2,n8zrrz,t5_2qwyh,1.620637e+09,It's too much (TW Abuse),venting going life well study university live ...,81.0,0.0,"[venting, going, life, well, study, university..."
3,o13rtw,t5_2qwyh,1.623847e+09,How to get through exam season when I am too d...,idk depressed seeing new therapist diagnosed f...,78.0,0.0,"[idk, depressed, seeing, new, therapist, diagn..."
4,qtqrwr,t5_2qwyh,1.636900e+09,Considering admitting myself to a ward,since years old thoughts parents going would l...,77.0,0.0,"[since, years, old, thoughts, parents, going, ..."


In [ ]:
#Some statistics
def get_word_count(text):
    return len(str(text).split())

df["word_count"] = df["selftext"].apply(get_word_count)

print(df["word_count"].max())
print(df["word_count"].min())
print(df["word_count"].mean())
print(df["word_count"].std())

AttributeError: 'float' object has no attribute 'split'

In [27]:
tagged_data = [TaggedDocument(words=row, tags=[str(i)]) for i, row in enumerate(df["tokenized_selftext"])]

doc2vec_model = Doc2Vec(vector_size=50, window=2, min_count=2, workers=4, epochs=40)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

In [28]:
doc_embeddings = [doc2vec_model.dv[str(i)] for i in range(len(tagged_data))]

In [29]:
num_clusters = 14
kmeans = KMeans(n_clusters=num_clusters)
df["cluster"] = kmeans.fit_predict(doc_embeddings)

group_by_cluster_14 = df.copy().groupby("cluster")

samples_clusters_14 = group_by_cluster_14.apply(lambda x: x.sample(n=min(len(x), 10), random_state=42))
samples_clusters_14 = samples_clusters_14.reset_index(drop=True)
samples_clusters_14.to_csv("clustered_data_14.csv")

/tmp/ipykernel_5113/2261839967.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  samples_clusters_14 = group_by_cluster_14.apply(lambda x: x.sample(n=min(len(x), 10), random_state=42))


In [30]:
num_clusters = 21
kmeans = KMeans(n_clusters=num_clusters)
df["cluster"] = kmeans.fit_predict(doc_embeddings)

group_by_cluster_21 = df.copy().groupby("cluster")

samples_clusters_21 = group_by_cluster_21.apply(lambda x: x.sample(n=min(len(x), 10), random_state=42))
samples_clusters_21 = samples_clusters_21.reset_index(drop=True)
samples_clusters_21.to_csv("clustered_data_21.csv")

/tmp/ipykernel_5113/170459378.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  samples_clusters_21 = group_by_cluster_21.apply(lambda x: x.sample(n=min(len(x), 10), random_state=42))


In [31]:
num_clusters = 28
kmeans = KMeans(n_clusters=num_clusters)
df["cluster"] = kmeans.fit_predict(doc_embeddings)

group_by_cluster_28 = df.copy().groupby("cluster")

samples_clusters_28 = group_by_cluster_28.apply(lambda x: x.sample(n=min(len(x), 10), random_state=42))
samples_clusters_28 = samples_clusters_28.reset_index(drop=True)
samples_clusters_28.to_csv("clustered_data_28.csv")

/tmp/ipykernel_5113/1065882364.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  samples_clusters_28 = group_by_cluster_28.apply(lambda x: x.sample(n=min(len(x), 10), random_state=42))


# Manually labeling data

In [37]:
class TOPIC(enum.Enum):
    ACADEMICS = 1
    CAREER = 2
    FAMILY = 3
    FINANCES = 4
    HEALTH = 5
    RELATIONS = 6
    OTHER = 7
    NONE = 8


In [38]:
clusters_14 = {0: [TOPIC.FINANCES, TOPIC.FAMILY], 1: [TOPIC.ACADEMICS, TOPIC.RELATIONS, TOPIC.HEALTH], 2: [TOPIC.HEALTH, TOPIC.OTHER, TOPIC.RELATIONS], 3: [TOPIC.]}